In [1]:
import agents_env
from agents_env.agent_replay_motion import HumanoidReplay
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *

In [2]:

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap

In [3]:
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'

In [4]:
data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

In [5]:
trajectory.all_states

[{'root_pos': array([0.      , 0.      , 0.755213]),
  'root_rot': array([-9.98753896e-01, -1.56329212e-02, -4.73889973e-02,  7.41752800e-04]),
  'chest': array([ 0.98463402, -0.02421333,  0.17156441, -0.02179934]),
  'neck': array([ 0.92988643, -0.10478597, -0.19867835,  0.29130402]),
  'right_hip': array([ 0.9313745 , -0.07018205, -0.27435029, -0.2287967 ]),
  'right_knee': array([-1.23311177]),
  'right_ankle': array([ 0.96711253, -0.11525968, -0.15977069, -0.16087852]),
  'right_shoulder': array([ 0.87983352, -0.18093009,  0.33238639,  0.28753532]),
  'right_elbow': array([1.76083421]),
  'left_hip': array([ 0.88979552,  0.08174648, -0.42077278,  0.15662605]),
  'left_knee': array([-0.97332998]),
  'left_ankle': array([9.99958596e-01, 7.65695400e-04, 9.03625060e-03, 7.51972600e-04]),
  'left_shoulder': array([ 0.90963906,  0.35224765, -0.18458766, -0.12002407]),
  'left_elbow': array([0.99051862])},
 {'root_pos': array([0.00697176, 0.00595309, 0.752527  ]),
  'root_rot': array([-0.

In [6]:
trajectory.data_xrot

[array([[-9.98753896e-01, -1.56329212e-02, -4.73889973e-02,
          7.41752800e-04],
        [-9.75639155e-01,  9.69624587e-03, -2.18370191e-01,
          1.86730320e-02],
        [-9.55042543e-01,  5.13475192e-02, -1.40023294e-02,
         -2.91652376e-01],
        [-7.89431571e-01,  1.16057757e-01, -5.22585116e-01,
         -3.00388395e-01],
        [-9.05648912e-01, -1.57691810e-01,  2.75857676e-01,
         -2.80777286e-01],
        [-9.28962190e-01, -3.05190022e-01, -1.08057948e-02,
          2.09216480e-01],
        [-8.22478762e-01, -1.69087880e-01,  4.31990974e-01,
          3.29122734e-01],
        [-9.44142540e-01,  6.65804367e-02,  2.26242700e-01,
          2.30165487e-01],
        [-9.01122091e-01, -7.87669747e-02, -3.61344534e-01,
          2.26284930e-01],
        [-9.01892999e-01,  1.21972783e-01, -2.44241368e-01,
          3.34750374e-01],
        [-9.07464981e-01, -1.02664959e-01,  3.80591094e-01,
         -1.45319074e-01],
        [-9.80101125e-01, -2.27823751e-02, 

In [7]:
trajectory.data_xpos

[array([[ 0.00000000e+00,  0.00000000e+00,  7.55213000e-01],
        [ 2.23485513e-02, -7.39086124e-03,  9.90187918e-01],
        [ 1.17831088e-01, -4.98068551e-03,  1.19268681e+00],
        [ 9.85563464e-02, -1.86739367e-01,  1.22561606e+00],
        [-1.09009048e-01, -3.23362951e-01,  1.10831709e+00],
        [ 1.10349194e-01,  1.79156382e-01,  1.21570053e+00],
        [ 1.39923312e-01,  3.36209123e-01,  9.92164730e-01],
        [-2.51546829e-04, -8.48454158e-02,  7.52568211e-01],
        [ 1.66917442e-01, -1.81745928e-01,  3.77913904e-01],
        [-8.53917774e-02, -5.65343456e-02,  8.01631114e-02],
        [ 2.51546829e-04,  8.48454158e-02,  7.57857789e-01],
        [ 2.78854598e-01,  2.10020971e-01,  4.67319580e-01],
        [ 2.04857127e-01,  2.15595493e-01,  6.42231700e-02]]),
 array([[ 6.97175780e-03,  5.95309350e-03,  7.52527000e-01],
        [ 2.92215588e-02,  6.61740263e-04,  9.87567938e-01],
        [ 1.24008770e-01,  2.92351341e-03,  1.19039494e+00],
        [ 1.04768303e-

Now I try to just compute the global pos of the chest

In [8]:
from utils import BODY_INTIAL_XPOS_MUJOCO_XML
from some_math.math_utils import *

#relative to the global
chest_global_pos = BODY_INTIAL_XPOS_MUJOCO_XML['chest']
#I got this differectly from the xml file this is relative to the root
chest_local_pos = np.array([0 ,0 ,0.236151])

In [9]:
root_initial_pos = BODY_INTIAL_XPOS_MUJOCO_XML['root']
#the first frame
root_current_pos = trajectory.data_xpos[1][0]
root_current_rot = trajectory.data_xrot[1][0]

In [10]:
print(root_current_pos)
print(root_current_rot)

[0.00697176 0.00595309 0.752527  ]
[-0.99882322 -0.0112808  -0.04714939 -0.00136157]


In [11]:
#get the local quat
#in the first frame
local_quat_chest = trajectory.all_states[1]['chest']
local_quat_chest

array([ 0.98496439, -0.019058  ,  0.17024069, -0.02236166])

In [12]:
#get the global quat
global_rotation = get_global_rotation_quat(root_current_rot,local_quat_chest)
global_rotation

array([-0.976024  ,  0.00921052, -0.21670713,  0.01817523])

In [13]:

#I will do it with transformations

rotation_parent_matrix = quaternion_to_rotation_matrix(root_current_rot)


transformation_matrix = apply_transformation(rotation_parent_matrix,root_current_pos,chest_local_pos)
global_pos_chest = transformation_matrix[:3] 
global_pos_chest



array([2.92215588e-02, 6.61740263e-04, 9.87567938e-01])

In [14]:

# Step 3: Calculate the global position of the chest by adding the rotated local position to the root's global position
rotated_local_pos_chest = rotate_vec_quat(chest_local_pos, root_current_rot)
rotated_local_pos_chest

global_pos_chest = root_current_pos + rotated_local_pos_chest
global_pos_chest

array([2.92215588e-02, 6.61740263e-04, 9.87567938e-01])

In [21]:
data_mocap_matrix.shape

(65, 44)

In [20]:
data_pos_mocap.shape 

(65, 35)